# Building a research Agent with LangChain

In [1]:
%pip install langchain
%pip install langchain-community
%pip install langchain-openai
%pip install langchainhub
%pip install beautifulsoup4

In [2]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# FILEPATH: /Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/1.3-intro-agents-with-langchain.ipynb
# SET YOUR API KEYS FOR LANGSMITH AND TAVILY TO RUN THIS NOTEBOOK!
# langsmith: https://docs.smith.langchain.com/#:~:text=install%20-U%20langsmith-,Create%20an%20API%20key,click%20Create%20API%20Key.,-Setup%20your%20environment
# tavily: https://app.tavily.com/sign-in
# openai: https://platform.openai.com/docs/quickstart
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your-api-key"
os.environ["TAVILY_API_KEY"] = "your-api-key"
os.environ["OPENAI_API_KEY"] = 'your-api-key'

# EXERCISE: Create a simple agent that performs a productive task, whatever that means for you.

In [38]:
from langchain_openai import ChatOpenAI

In [39]:
llm_chat = ChatOpenAI(temperature=0)

In [40]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import tool

search = TavilySearchResults()

results = search.invoke("biggest moutains in the world")
results

[{'url': 'https://en.wikipedia.org/wiki/List_of_highest_mountains_on_Earth',
  'content': "88 | 4\n87 |\n86 | 1\n85 | 8 1\n84 | 8\n83 |\n82 |\n81 | 8 6 6 2\n80 | 9 8 5 3 2\n79 | 5 4 3 3\n78 | 9 8 7 6 2 2 1 0\n77 | 9 9 8 8 5 5 4 1 0 0\n76 |\n9 7 6 6 4 1 1\n75 | 7 7 5 5 4 4 4 3 3 1 1\n74 | 9 9 9 9 7 6 6 6 5 5 3 3 2 2 2 1 1 1 0 0 0\n73 | 8 8 8 8 8 6 6 5 5 5 4 2 1 1 1 0\n72 | 9 9 9 8 8 8 8 7 7 6 6 5 4 4 4 3 2 2 1 1 0 0 0 0 0 0 0\nGallery[edit]\n1. All peaks 7,000\xa0m (23,000\xa0ft) or higher are located in East, Central or South Asia in a rectangle edged by Noshaq (7,492\xa0m or 24,580\xa0ft) on the Afghanistan–Pakistan border in the west, Jengish Chokusu (Tuōmù'ěr Fēng, 7,439\xa0m or 24,406\xa0ft) on the Kyrgyzstan–Xinjiang border to the north, Gongga Shan (Minya Konka, 7,556\xa0m or 24,790\xa0ft) in Sichuan to the east, and Kabru (7,412\xa0m or 24,318\xa0ft) on the Sikkim–Nepal border to the south.\n Mauna Loa (4,169\xa0m or 13,678\xa0ft) is the largest mountain on Earth in terms of bas

In [41]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import tool

search = TavilySearchResults()

@tool
def web_search(query: str):
    """Searches the web given a user query and returns the most relevant links"""
    output = search.invoke(query)
    return [r['url'] for r in output if type(r)==dict]

web_search.invoke("Current resources on why pancakes are the best breakfast")

['https://www.medicinenet.com/are_pancakes_healthy_or_unhealthy_breakfast_food/article.htm',
 'https://www.eatingwell.com/article/17772/which-is-a-healthier-breakfast-pancakes-or-eggs/',
 'https://www.chefsresource.com/faq/is-pancakes-good-for-breakfast/',
 'https://www.popsugar.com/food/why-pancakes-better-than-waffles-46335811',
 'https://fitonapp.com/nutrition/healthy-pancakes/']

In [42]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")

prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold

In [43]:
prompt.input_variables

['agent_scratchpad', 'input']

In [44]:
prompt.input_types

{'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]],
 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}

In [45]:
prompt.__dict__

{'name': None,
 'input_variables': ['agent_scratchpad', 'input'],
 'input_types': {'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]],
  'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]},
 'output_parser': None,
 'partial_variables': {},
 'metadata': {'lc_hub_owner': 'hwchase17',
  'lc_hub_repo': 'openai-tools-agent',
  'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'},
 'tags': None,
 'messages': [SystemMessagePromptTemplate(prompt=PromptTempl

To build an agent I need to do 2 essential things:

1. Connect a model to tools
2. Set up a loop in which the model continuously tries to solve the problem by either just calling the llm or calling a tool, observing the result and repeating until finished.

In [46]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold

In [47]:
from langchain.agents import create_tool_calling_agent

tools = [web_search]

agent = create_tool_calling_agent(llm_chat, tools, prompt)

In [48]:
from langchain.agents import AgentExecutor


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [49]:
agent_executor.invoke({"input": "What are good resources to learn about learning new languages?",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...

Invoking: `web_search` with `{'query': 'resources to learn new languages'}`


['https://www.fluentu.com/blog/learn/free-language-learning-websites/', 'https://nymag.com/strategist/article/how-to-learn-languages-at-home.html', 'https://www.businessinsider.com/guides/learning/best-online-apps-courses-to-learn-new-language?op=1', 'https://www.theintrepidguide.com/top-language-learning-resources/', 'https://www.pcmag.com/picks/the-best-language-learning-software']Here are some resources to learn new languages:

1. [FluentU - Free Language Learning Websites](https://www.fluentu.com/blog/learn/free-language-learning-websites/)
2. [NYMag - How to Learn Languages at Home](https://nymag.com/strategist/article/how-to-learn-languages-at-home.html)
3. [Business Insider - Best Online Apps and Courses to Learn New Language](https://www.businessinsider.com/guides/learning/best-online-apps-courses-to-learn-new-language?op=1)
4. [The Intrepid Guide - Top Languag

{'input': 'What are good resources to learn about learning new languages?',
 'agent_scratchpad': [],
 'output': 'Here are some resources to learn new languages:\n\n1. [FluentU - Free Language Learning Websites](https://www.fluentu.com/blog/learn/free-language-learning-websites/)\n2. [NYMag - How to Learn Languages at Home](https://nymag.com/strategist/article/how-to-learn-languages-at-home.html)\n3. [Business Insider - Best Online Apps and Courses to Learn New Language](https://www.businessinsider.com/guides/learning/best-online-apps-courses-to-learn-new-language?op=1)\n4. [The Intrepid Guide - Top Language Learning Resources](https://www.theintrepidguide.com/top-language-learning-resources/)\n5. [PCMag - The Best Language Learning Software](https://www.pcmag.com/picks/the-best-language-learning-software)\n\nYou can explore these resources to find the best fit for your language learning journey.'}

In [15]:
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm_chat, tools, prompt)

react_agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

react_agent_executor.invoke({"input": "What is the best way to learn good programming practices?",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...


Learning good programming practices is essential for becoming a successful programmer.
Action: web_search
Action Input: "best way to learn good programming practices"

['https://www.datacamp.com/tutorial/coding-best-practices-and-guidelines', 'https://www.educative.io/blog/coding-best-practices', 'https://stackoverflow.com/questions/1301606/how-can-i-practice-better-object-oriented-programming', 'https://www.freecodecamp.org/news/learn-coding-for-everyone-handbook/', 'https://www.freecodecamp.org/news/how-to-learn-programming/']

There are several resources available online to learn good programming practices.
Action: web_search
Action Input: "coding best practices tutorial"

['https://www.datacamp.com/tutorial/coding-best-practices-and-guidelines', 'https://www.freecodecamp.org/news/clean-coding-for-beginners/', 'https://www.freecodecamp.org/news/how-to-write-clean-code/', 'https://realpython.com/tutorials/best-practices/', 'https://www.datacamp.com/blog/python-best-practices-for-better-code']

I should explore these tutorials and articles to understand and practice good programming practices.
Final Answer: The best way to learn good programming practices is to explore tutorials and articles available online that cover coding best practices.

> Finished chain.


{'input': 'What is the best way to learn good programming practices?',
 'agent_scratchpad': [],
 'output': 'The best way to learn good programming practices is to explore tutorials and articles available online that cover coding best practices.'}

Now let's build this same agent but using a more in depth approach going one level down in the abstraction layers.

Now, let's go down the latter of abstraction and build this same agent but leveraging the lower-level capabilities of LangChain's LCEL language. 

In [16]:
from langchain_openai import ChatOpenAI

In [17]:
llm_chat = ChatOpenAI(model="gpt-4o",temperature=0)

In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import tool

search = TavilySearchResults()

@tool
def web_search(query: str):
    """Searches the web given a user query and returns the most relevant links"""
    output = search.invoke(query)
    return [r['url'] for r in output if type(r)==dict]


tools = [web_search]

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a powerful research assistant.\
            Given a user query you will perform research on the \
                topic using the tools available."
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Bind tools to the LLM

In [21]:
llm_with_tools = llm_chat.bind_tools(tools)

Now, all we have to do is import two last utility functions:
- a component for formatting intermediate steps (agent action, tool output pairs) to input messages that can be sent to the model, 
- and a component for converting the output message into an agent action/agent finish.

In [22]:
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.output_parsers import ToolsAgentOutputParser

In [23]:
agent = (
    {
        "input": lambda x: x["input"], 
        "agent_scratchpad": lambda x: format_to_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | ToolsAgentOutputParser()
)

In [24]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
list(agent_executor.stream({"input": "What are some good resources about open source large language models?"}))



> Entering new AgentExecutor chain...



Invoking: `web_search` with `{'query': 'open source large language models resources'}`




['https://llama.meta.com/llama2/', 'https://llama.meta.com/', 'https://www.datacamp.com/blog/top-open-source-llms', 'https://ai.meta.com/blog/meta-llama-3/', 'https://github.com/Hannibal046/Awesome-LLM']
Invoking: `web_search` with `{'query': 'best open source large language models 2023'}`




['https://cheatsheet.md/llm-leaderboard/best-open-source-llm.en', 'https://huggingface.co/blog/2023-in-llms', 'https://www.shakudo.io/blog/comparing-opensource-large-language-models', 'https://www.clarifai.com/blog/top-10-open-source-large-language-models-in-2023', 'https://www.lakera.ai/blog/open-source-llms']

Here are some good resources about open source large language models:

1. **Meta's LLaMA (Large Language Model Meta AI)**
   - [LLaMA 2](https://llama.meta.com/llama2/)
   - [LLaMA](https://llama.meta.com/)
   - [Meta AI Blog on LLaMA 3](https://ai.meta.com/blog/meta-llama-3/)

2. **DataCamp Blog**
   - [Top Open Source LLMs](https://www.datacamp.com/blog/top-open-source-llms)

3. **GitHub Repository**
   - [Awesome LLM](https://github.com/Hannibal046/Awesome-LLM)

4. **CheatSheet.md**
   - [LLM Leaderboard: Best Open Source LLM](https://cheatsheet.md/llm-leaderboard/best-open-source-llm.en)

5. **Hugging Face Blog**
   - [2023 in LLMs](https://huggingface.co/blog/2023-in-llms)

6. **Shakudo Blog**
   - [Comparing Open Source Large Language Models](https://www.shakudo.io/blog/comparing-opensource-large-language-models)

7. **Clarifai Blog**
   - [Top 10 Open Source Large Language Models in 2023](https://www.clarifai.com/blog/top-10-open-source-large-language-models-in-2023)

8. **Laker

[{'actions': [ToolAgentAction(tool='web_search', tool_input={'query': 'open source large language models resources'}, log="\nInvoking: `web_search` with `{'query': 'open source large language models resources'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_azX6KCUwWb6kB2TpvAvG6Jem', 'function': {'arguments': '{"query": "open source large language models resources"}', 'name': 'web_search'}, 'type': 'function'}, {'index': 1, 'id': 'call_32hx2DuacrlXsmQmh5r2qTwm', 'function': {'arguments': '{"query": "best open source large language models 2023"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c4e5b6fa31'}, id='run-519eeb64-5938-4b64-9918-0af8e5034324', tool_calls=[{'name': 'web_search', 'args': {'query': 'open source large language models resources'}, 'id': 'call_azX6KCUwWb6kB2TpvAvG6Jem', 'type': 'tool_call'}, {'name

Now, what if we want the agent to actually read the links and return a research report rather than just the urls?

In [26]:
import requests
from bs4 import BeautifulSoup


@tool
def scrape_text(url: str):
    """Scrapes the text from a given URL and returns it."""
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text[:10000]
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"

In [27]:
tools = [web_search, scrape_text]

In [28]:
llm_with_tools = llm_chat.bind_tools(tools)

In [29]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [30]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a powerful research assistant. Given a user query you will perform research on the topic using the tools available."
        ),
        ("user", "Scrape and summarize the contents from these urls: {input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [31]:
agent = (
    {
        "input": lambda x: x["input"], 
        "agent_scratchpad": lambda x: format_to_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | ToolsAgentOutputParser()
)

In [32]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [33]:
agent_executor.invoke({"input": "How to build a simple research agent using langchain in Python?"})



> Entering new AgentExecutor chain...



Invoking: `web_search` with `{'query': 'How to build a simple research agent using langchain in Python'}`




['https://python.langchain.com/v0.2/docs/tutorials/agents/', 'https://towardsdatascience.com/building-a-simple-agent-with-tools-and-toolkits-in-langchain-77e0f9bd1fa5', 'https://dev.to/timesurgelabs/how-to-make-an-ai-agent-in-10-minutes-with-langchain-3i2n', 'https://build5nines.com/introduction-to-building-ai-agents-with-langchain-and-python/', 'https://analyzingalpha.com/langchain-python-tutorial']
Invoking: `web_search` with `{'query': 'Langchain research agent tutorial'}`




['https://python.langchain.com/v0.2/docs/tutorials/agents/', 'https://medium.com/@ranadevrat/understanding-langchain-agents-a-beginners-guide-8a87708dc48e', 'https://dev.to/timesurgelabs/how-to-make-an-ai-agent-in-10-minutes-with-langchain-3i2n', 'https://js.langchain.com/v0.2/docs/tutorials/agents/', 'https://blog.langchain.dev/how-to-build-the-ultimate-ai-automation-with-multi-agent-collaboration/']


Invoking: `scrape_text` with `{'url': 'https://python.langchain.com/v0.2/docs/tutorials/agents/'}`


Build an Agent | 🦜️🔗 LangChain Skip to main content LangChain 0.2 is out! Leave feedback on the v0.2 docs here . You can view the v0.1 docs here . Integrations API Reference More People Contributing Templates Cookbooks 3rd party tutorials YouTube arXiv v0.2 v0.2 v0.1 🦜️🔗 LangSmith LangSmith Docs Templates GitHub Templates Hub LangChain Hub JS/TS Docs 💬 Search Introduction Tutorials Build a Question Answering application over a Graph Database Tutorials Build a Simple LLM Application with LCEL Build a Query Analysis System Build a Chatbot Conversational RAG Build an Extraction Chain Build an Agent Tagging data_generation Build a Local RAG Application Build a PDF ingestion and Question/Answering system Build a Retrieval Augmented Generation (RAG) App Vector stores and retrievers Build a Question/Answering system over SQL data Summarize Text How-to guides How-to guides How to use tools in 

Building a simple Agent with Tools and Toolkits in LangChain | by Sami Maameri | Towards Data Science Open in app Sign up Sign in Write Sign up Sign in Building a simple Agent with Tools and Toolkits in LangChain Get familiar with the building blocks of Agents in LangChain Sami Maameri · Follow Published in Towards Data Science · 13 min read · Apr 10, 2024 -- 1 Listen Share Photo by Dan LeFebvre on Unsplash Let’s build a simple agent in LangChain to help us understand some of the foundational concepts and building blocks for how agents work there. By keeping it simple we can get a better grasp of the foundational ideas behind these agents, allowing us to build more complex agents in the future. Contents What are Agents? Building the Agent - The Tools - The Toolkit - The LLM - The Prompt The Agent Testing our Agent Observations The Future Conclusion Want to get notified about my future posts? Subscribe here What are Agents The LangChain documentation actually has a pretty good page on t

Introduction To Building AI Agents With LangChain And Python | Build5Nines Newsletter Advertise Tools IPv4 Address CIDR Range Reference and Calculator Byte Conversion Calculator from KB, MB, GB, TB, PB More Cloud DevOps CI / CD Artificial Intelligence Certification Internet of Things Blockchain Community Development Hardware Operating Systems Cloud, DevOps & SRE Guidance Cloud Microsoft Azure Amazon AWS Google Cloud Platform DevOps HashiCorp Terraform Azure Bicep ARM Templates Azure CLI PowerShell CI / CD Azure DevOps GitHub Git AI Microsoft OpenAI Development C# Node.js Python Certification Top DevOps and SRE Certifications Microsoft Certification HashiCorp Certification GitHub Certification Development • Python Introduction to Building AI Agents with LangChain and Python By Chris Pietschmann June 23, 2024  7:07 AM EDT LangChain is a tool for more easily creating AI agents that can autonomously perform tasks. These agents can streamline operations, enhance user experiences, and handle

LangChain Python Tutorial: The Ultimate Step-by-Step Guide - Analyzing Alpha Skip to content Analyzing Alpha Analyzing Alpha Menu Home About Blog Podcast Recommended Menu Home About Blog Podcast Recommended Programming LangChain Python Tutorial: The Ultimate Step-by-Step Guide By Leo Smigel Updated on October 13, 2023 As a Python programmer, you might be looking to incorporate large language models (LLMs) into your projects – anything from text generators to trading algorithms. That’s where this comprehensive LangChain Python guide comes in, tailored to fit both novices and seasoned coders. Unlike dense official documents or confusing tutorials, I bring a simplified approach to this tutorial, drawing from years of experience in making complex concepts more digestible. With easy-to-follow instructions and lucid examples, I’ll guide you through the intricate world of LangChain, unlocking its immense potential. Don’t delay; start leveraging LangChain to build innovative applications today

Here is a summary of the contents from the scraped URLs on how to build a simple research agent using LangChain in Python:

1. **LangChain Documentation: Build an Agent**
   - **Overview**: This guide provides a comprehensive tutorial on building an agent that can interact with a search engine using LangChain.
   - **Components**:
     - **Chat Models**: Used as reasoning engines.
     - **Tools**: Methods or classes that the agent can use to interact with its environment.
     - **Agents**: Systems that use LLMs to determine actions and inputs.
   - **Example**: The tutorial includes a code snippet for creating an agent with conversational memory and a search tool.
   - **Steps**:
     1. Import necessary modules.
     2. Create the agent with memory and tools.
     3. Use the agent to interact with the environment.

2. **Towards Data Science: Building a Simple Agent with Tools and Toolkits in LangChain**
   - **Overview**: This article explains the foundational concepts and building 

{'input': 'How to build a simple research agent using langchain in Python?',
 'output': "Here is a summary of the contents from the scraped URLs on how to build a simple research agent using LangChain in Python:\n\n1. **LangChain Documentation: Build an Agent**\n   - **Overview**: This guide provides a comprehensive tutorial on building an agent that can interact with a search engine using LangChain.\n   - **Components**:\n     - **Chat Models**: Used as reasoning engines.\n     - **Tools**: Methods or classes that the agent can use to interact with its environment.\n     - **Agents**: Systems that use LLMs to determine actions and inputs.\n   - **Example**: The tutorial includes a code snippet for creating an agent with conversational memory and a search tool.\n   - **Steps**:\n     1. Import necessary modules.\n     2. Create the agent with memory and tools.\n     3. Use the agent to interact with the environment.\n\n2. **Towards Data Science: Building a Simple Agent with Tools and T

We encounter a few issues when we try to just add the scrape function to the agent without any other consideration. 
How about we make this better by improving the inner chain of the agent?

In [34]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | ToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

input1 = "What are good resources on learning about the langchain framework?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)



> Entering new AgentExecutor chain...



Invoking: `web_search` with `{'query': 'LangChain framework learning resources'}`




['https://medium.com/@ankity09/top-5-resources-to-learn-langchain-e2bdbbd11702', 'https://python.langchain.com/v0.2/docs/introduction/', 'https://medium.com/@ellen.hoeven/introducing-langchain-a-beginners-guide-️-5e751622edb7', 'https://www.freecodecamp.org/news/beginners-guide-to-langchain/', 'https://medium.com/@zilliz_learn/ultimate-guide-to-getting-started-with-langchain-b9a87cb340f8']


Invoking: `scrape_text` with `{'url': 'https://medium.com/@ankity09/top-5-resources-to-learn-langchain-e2bdbbd11702'}`


Top 5 Resources to learn LangChain | by Ankit Yadav | Medium Open in app Sign up Sign in Write Sign up Sign in Top 5 Resources to learn LangChain Ankit Yadav · Follow 3 min read · Jul 8, 2023 -- Listen Share Photo by Karine Avetisyan on Unsplash Large Language Models(LLMs) have taken the world by storm ever since OpenAI announced the launch of ChatGPT. ChatGPT holds the record for fastest to 100 million Daily Active Users(DAU) which was achieved in 2 months after its launch. Meta’s Threads App is on track to break this record, but it does have the advantage of a seamless integration for Instagram users. OpenAI and other open-source LLMs are here to stay,bringing with them a new wave of applications and use cases. One crucial foundation for these applications is LangChain, a Python framework for building Language Model Applications. The highlight of the framework is 

Introduction | 🦜️🔗 LangChain Skip to main content LangChain 0.2 is out! Leave feedback on the v0.2 docs here . You can view the v0.1 docs here . Integrations API Reference More People Contributing Templates Cookbooks 3rd party tutorials YouTube arXiv v0.2 v0.2 v0.1 🦜️🔗 LangSmith LangSmith Docs Templates GitHub Templates Hub LangChain Hub JS/TS Docs 💬 Search Introduction Tutorials Build a Question Answering application over a Graph Database Tutorials Build a Simple LLM Application with LCEL Build a Query Analysis System Build a Chatbot Conversational RAG Build an Extraction Chain Build an Agent Tagging data_generation Build a Local RAG Application Build a PDF ingestion and Question/Answering system Build a Retrieval Augmented Generation (RAG) App Vector stores and retrievers Build a Question/Answering system over SQL data Summarize Text How-to guides How-to guides How to use tools in a chain How to use a vectorstore as a retriever How to add memory to chatbots How to use example selecto

Introducing LangChain: a beginner’s guide 🦜️🔗 | by Ellen Hoeven | Medium Open in app Sign up Sign in Write Sign up Sign in Introducing LangChain: a beginner’s guide 🦜️🔗 Build applications with LLMs using LangChain Ellen Hoeven · Follow 6 min read · Sep 22, 2023 -- 1 Listen Share Have you heard the term LangChain but not sure what exactly it is about? Then this is your chance to catch up: this article provides an overview of the most important concepts in 5 minutes, explaining everything you need to know to continue your journey with LangChain. Why LangChain? Along with the recent developments in large language models (LLMs), many tools and libraries have risen around them to ease the development of LLM applications. LangChain is an open-source development framework for building LLM applications. It bundles common functionalities that are needed for the development of more complex LLM projects. “Building applications with LLMs through composability.” It does so by providing modular comp

How to Use LangChain to Build With LLMs – A Beginner's Guide Search Submit your search query Forum Donate April 11, 2024 / #AI How to Use LangChain to Build With LLMs – A Beginner's Guide Jacob Lee Large language models (LLMs) are incredibly powerful general reasoning tools that are useful in a wide range of situations. But working with LLMs presents challenges that are different from building traditional software: Calls tend to be long-running, and stream generated output as it becomes available. Instead of structured input (something like JSON) with fixed parameters, they take unstructured and arbitrary natural language as input. They are capable of “understanding” subtleties of that language. They are non-deterministic. You may get different outputs even with the same input. LangChain is a popular framework for creating LLM-powered apps. It was built with these and other factors in mind, and provides a wide range of integrations with closed-source model providers (like OpenAI , Anth

Ultimate Guide to Getting Started with LangChain | by Zilliz | Medium Open in app Sign up Sign in Write Sign up Sign in Ultimate Guide to Getting Started with LangChain Zilliz · Follow 6 min read · Nov 17, 2023 -- Listen Share Have you seen the parrot + chain emoji popping up around AI lately? Those are LangChain’s signature emojis. LangChain is an AI Agent tool that adds functionality to large language models (LLMs) like GPT. In addition, it includes functionality such as token management and context management. For this getting started tutorial, we look at two primary LangChain examples with real-world use cases. First, how to query GPT. Second, how to query a document with a Colab notebook available here . In this tutorial we cover: What is LangChain? How Can You Run LangChain Queries? Query GPT Query a Document Introduction to LangChain Summary What is LangChain? LangChain is a framework for building applications that leverage LLMs. It allows you to quickly build with the CVP Frame

Here are some excellent resources to learn about the LangChain framework:

1. **[Top 5 Resources to Learn LangChain](https://medium.com/@ankity09/top-5-resources-to-learn-langchain-e2bdbbd11702)** by Ankit Yadav on Medium:
   - This article provides a curated list of valuable resources, including short courses by DeepLearning.ai, articles, YouTube playlists, and the official LangChain tutorials page.

2. **[LangChain Official Documentation](https://python.langchain.com/v0.2/docs/introduction/)**:
   - The official documentation is a comprehensive resource that covers everything from basic concepts to advanced tutorials. It includes guides on building various applications, how-to guides, and an API reference.

3. **[Introducing LangChain: A Beginner’s Guide](https://medium.com/@ellen.hoeven/introducing-langchain-a-beginners-guide-️-5e751622edb7)** by Ellen Hoeven on Medium:
   - This article provides an overview of the most important concepts in LangChain and includes hands-on examples 

In [35]:
result

{'input': 'What are good resources on learning about the langchain framework?',
 'chat_history': [HumanMessage(content='What are good resources on learning about the langchain framework?'),
  AIMessage(content="Here are some excellent resources to learn about the LangChain framework:\n\n1. **[Top 5 Resources to Learn LangChain](https://medium.com/@ankity09/top-5-resources-to-learn-langchain-e2bdbbd11702)** by Ankit Yadav on Medium:\n   - This article provides a curated list of valuable resources, including short courses by DeepLearning.ai, articles, YouTube playlists, and the official LangChain tutorials page.\n\n2. **[LangChain Official Documentation](https://python.langchain.com/v0.2/docs/introduction/)**:\n   - The official documentation is a comprehensive resource that covers everything from basic concepts to advanced tutorials. It includes guides on building various applications, how-to guides, and an API reference.\n\n3. **[Introducing LangChain: A Beginner’s Guide](https://mediu